In [1]:
import pandas as pd
import numpy as np
import glob
import matplotlib as mp
from functools import reduce


In [2]:
#Import all csv files of VCF
files = glob.glob('G:\\Dropbox (Vetsigian lab)\\Vetsigian lab Team Folder\\Ye\\Genomics\\M145 Evolved mutants\\*.csv')
strain_names = list(map(lambda f: f.split('\\')[6].split('_')[0], files))
vcfs = dict()
for (strain, file) in zip(strain_names, files):
    vcf = pd.read_csv(file)
    vcf['Variant Frequency']= list(map(lambda x: float(x[0].replace('%', ''))/100, vcf['Variant Frequency'].str.split( '->')))
    vcf.sort_values(by = ['Variant Frequency'], ascending = False, inplace = True)
    vcf['locus_tag'].fillna('intergenetic region', inplace = True)
    vcfs[strain] = vcf[['Name', 'Minimum', 'Maximum', 'Sequence', 'Variant Frequency', 'locus_tag', 'note']]
    
vcfs['R1'].head(5)


,Name,Minimum,Maximum,Sequence,Variant Frequency,locus_tag,note
0,G,619172,619172,C,1.0,SCO0577,"SCF55.01c, hypothetical protein, len: >231 aa;..."
22,G,4397330,4397330,C,1.0,intergenetic region,NaN
24,G,4726818,4726818,C,1.0,SCO4315,"SCD95A.48, possible homeostasis protein, len: ..."
25,G,4912786,4912785,-,1.0,SCO4493,"SCD69.13, probable asnC-family transcriptional..."
26,G,5107398,5107398,C,1.0,SCO4676,"SCD31.01c, unknown (fragment), len: >147 aa; S..."


In [3]:
#Find all variance in mutants with 100% Variant Frquency (Fixed mutations)
def find_fixed_mutants(vcfs):
    '''
    input: vcfs, dictionary of variant files of all mutants and WT compared to the ref genome;
           
    output: dictionary of variant files of mutants with 100% variant freq, the WT file is unchanged
    '''
    vcfs_fixed =  dict()
    for key in vcfs:
        vcf = vcfs[key]        
        if key != 'WT':
#             vcfs_fixed[key] = vcf[vcf['Variant Frequency'] == 1].set_index([ 'Minimum', 'Maximum', 'Name'])
            vcfs_fixed[key] = vcf[vcf['Variant Frequency'] == 1].reset_index().drop('index', axis = 1)
        else:
#             vcfs_fixed[key] = vcf.set_index(['Minimum', 'Maximum', 'Name'])
            vcfs_fixed[key] = vcf.reset_index().drop('index', axis = 1)
    return vcfs_fixed


In [4]:
vcfs_fixed = find_fixed_mutants(vcfs)
# find num of fixed variance in each strain
var_per_strain = list(map(lambda strain: len(vcfs_fixed[strain]), strain_names))

In [48]:
print(len(vcfs_fixed['R1']))
vcfs_fixed['R1'].head(5)

43


,Name,Minimum,Maximum,Sequence,Variant Frequency,locus_tag,note
0,G,619172,619172,C,1.0,SCO0577,"SCF55.01c, hypothetical protein, len: >231 aa;..."
1,G,4397330,4397330,C,1.0,intergenetic region,NaN
2,G,4726818,4726818,C,1.0,SCO4315,"SCD95A.48, possible homeostasis protein, len: ..."
3,G,4912786,4912785,-,1.0,SCO4493,"SCD69.13, probable asnC-family transcriptional..."
4,G,5107398,5107398,C,1.0,SCO4676,"SCD31.01c, unknown (fragment), len: >147 aa; S..."


In [49]:
# Find list of all fixed variance (with 100% variance frequency) across mutant strains, pick up the 'Minimum', 'Maximum' and 'Name' as the mark for the variance
def find_all_fixed_variance(vcfs_fixed, strain_names):
    '''
    input: vcfs_fixed by find_fixed_mutants()
    output: a combined data frame of all nonredundant variances across all mutants;
    '''
    df = vcfs_fixed[strain_names[0]]
    colnames = ['Minimum', 'Maximum', 'Name','Sequence', 'Variant Frequency','locus_tag', 'note']
    for strain in strain_names[1:-1]:
        df = pd.merge(df, vcfs_fixed[strain],  on = colnames, how = 'outer')
#         df = df.join(vcfs_fixed[strain], how='outer', on = ['Minimum', 'Maximum', 'Name'])

    return df

In [7]:
variances = find_all_fixed_variance(vcfs_fixed, strain_names).sort_values(by=['Minimum', 'Maximum']).reset_index().drop('index', axis=1)
print(len(variances))
variances.head(5)

116


,Name,Minimum,Maximum,Sequence,Variant Frequency,locus_tag,note
0,G,104233,104233,A,1.0,SCO0124,"SCJ21.05, unknown, len: 453 aa; similar in par..."
1,G,603360,603360,A,1.0,intergenetic region,NaN
2,G,619172,619172,C,1.0,SCO0577,"SCF55.01c, hypothetical protein, len: >231 aa;..."
3,G,657081,657081,T,1.0,SCO0617,"SCF56.01c, hypothetical protein (partial CDS),..."
4,C,676029,676028,-,1.0,intergenetic region,NaN


In [40]:
allmuts = variances[['Name', 'Minimum', 'Maximum']]
tab = variances.loc[:, ['locus_tag', 'Name', 'Minimum', 'Maximum']]
for strain in strain_names:
    tab[strain] = 0
tab['WT VQ'] = 0

In [69]:
def table_for_fixed_variance(vcfs_fixed, tab, strain_names, allmuts):  
    for strain in strain_names:
        vcf = vcfs_fixed[strain]
        vcf_muts = vcf[['Name', 'Minimum', 'Maximum']]
        bool1 = allmuts['Name'].isin(vcf['Name']) + 0
        bool2 = allmuts['Minimum'].isin(vcf['Minimum']) + 0
        bool3 = allmuts['Maximum'].isin(vcf['Maximum']) + 0
        mask_vcf = bool1+bool2+bool3 == 3
        tab.loc[mask_vcf, strain] = 1
        if strain == 'WT':
            tab.loc[mask_vcf, 'WT VQ'] = vcf.loc[mask_vcf, 'Variant Frequency']
    return tab

In [71]:
tab_variance = table_for_fixed_variance(vcfs_fixed, tab, strain_names, allmuts)
tab_variance.head(20)
save_path = 'G:\\Dropbox (Vetsigian lab)\\Vetsigian lab Team Folder\\Ye\\Genomics\\Comunity mutants\\table_fixed_variance_mutants.csv'
tab_variance.to_csv(save_path)

In [72]:
tab_variance.groupby('locus_tag').sum()


,Minimum,Maximum,R1,R2,R3,R4,R5,RH1,RH2,RH5,W1,W2,W3,W4,W5,WS1,WS2,WT,WT VQ
locus_tag,,,,,,,,,,,,,,,,,,,
SCO0124,104233,104233,0,1,0,1,1,0,0,1,1,1,1,1,1,0,1,1,1.000
SCO0577,619172,619172,1,1,0,1,1,0,0,1,1,0,0,0,0,1,1,1,1.000
SCO0617,657081,657081,1,1,1,1,1,0,0,1,0,1,1,0,1,0,1,1,1.000
SCO0676,719015,719015,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0.000
SCO1182,1246835,1246835,1,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0.000
SCO1337,1415134,1415134,1,0,0,1,1,0,0,1,1,1,1,1,0,1,1,0,0.000
SCO1428,1523375,1523376,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,1.000
SCO1483,1587068,1587068,0,0,0,1,0,1,0,1,1,0,1,1,1,0,1,1,1.000
SCO1511,1616266,1616265,0,0,1,1,1,0,1,1,0,1,1,1,1,0,0,1,1.000


In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure()
ax = fig.add_subplot(1,1,1)
tab_variance['R1'] == 1
#tab_variance['R1'].loc[tab_variance['R1'] == 1]
# for strain in strain_names:
#     loc = tab_variance['strain'] == 1
#     muts = tab_variance['strain'].loc[tab_variance['strain'] == 1]
    

In [ ]:
# save variance_table as csv
filepath = "G:\\Ye\\Evolution mutants\\variance_table.csv"
cols = variance_table.columns.values
df = variance_table.loc[:, cols[:-2]]
df.to_csv(filepath)
df

In [ ]:
import matplotlib.pyplot as plt
from matplotlib_venn import venn2

In [ ]:
def venn_WT_Mutant(variance_table, mutant_name):
    comp_table = variance_table[[mutant_name, 'WT']]
    strain_num = comp_table.aggregate('sum')
    common_diff_num = sum(comp_table.sum(axis = 1) == 2)
    venn2(subsets = (strain_num[mutant_name], strain_num['WT'], common_diff_num), set_labels = (mutant_name, 'WT'))
    plt.show()

In [ ]:
venn_WT_Mutant(variance_table, 'R5')

In [ ]:
from scipy.spatial.distance import squareform, pdist
from scipy.cluster.hierarchy import dendrogram
from scipy.cluster.hierarchy import linkage
cols = variance_table.columns.values
df = variance_table.loc[:, cols[0:-2]].transpose()
Y = pdist(df, 'euclidean')
dist_mat = squareform(Y)
linkage_matrix = linkage(dist_mat, "single")
dendrogram(linkage_matrix, color_threshold=1, labels=cols[0:-2],show_leaf_counts=True, leaf_rotation = 45)
plt.title=("test")
plt.show()

In [ ]:
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr
base = importr("base")
pvclust = importr("pvclust")
data = robjects.DataFrame.from_csvfile("G:\\Ye\\Evolution mutants\\variance_table.csv")
#result = pvclust.pvclust(df, nboot=100, method_dist="correlation", method_hclust="average")

data.head(5)
cols = robjects.IntVector(range(2,17))

subset = data.rx(True, cols)
subset.head(5)
result = pvclust.pvclust(subset, nboot=1000, method_dist="correlation", method_hclust="single")


In [ ]:
graphics = importr("graphics")
grdevices = importr("grDevices")
#grdevices.pdf("G:\\Ye\\Evolution mutants\\evolveMutants_phylo_boot1000.pdf", paper="a4")
graphics.plot(result)
#grdevices.dev_off()

In [ ]:
def find_variance_by_mutant(variance_table, mutant_name):
    variance = variance_table[mutant_name].where(variance_table[mutant_name] == 1).dropna().index.values
    variance = vcfs_100[mutant_name].loc[variance, ['locus_tag', 'note']]
    variance_genes = variance.dropna().reset_index(drop = True).set_index('locus_tag').drop_duplicates()
    return variance, variance_genes



In [ ]:
# Only look at genes
variance_full, variance_genes = find_variance_by_mutant(variance_table, 'WT') 
variance_full

In [ ]:
def find_hot_variance(variance_table, hot_thresh):
    cols = variance_table.columns.values
    df = variance_table.loc[:, cols[:-2]]
    hits = df.sum(axis = 1).sort_values(ascending = False) 
    hot_variance = hits[hits >= hot_thresh].index.values
    return hot_variance
  

In [ ]:
def find_hot_variance_by_mutant(variance_table, mutant_name, hot_thresh):
    hots = find_hot_variance(variance_table,hot_thresh)
    variance_full, variance_genes = find_variance_by_mutant(variance_table, mutant_name) 
    var_mutant = variance_full.index.values
    intersection = list(set(hots).intersection(var_mutant))
    hots_mut = variance_full.loc[intersection, ['locus_tag', 'note']]
    return hots_mut.dropna()[['locus_tag', 'note']].drop_duplicates().sort_values(by = 'locus_tag')

In [ ]:
find_hot_variance_by_mutant(variance_table, 'W2', 10)

In [ ]:
def find_cold_variance(variance_table, cold_thresh):
    cols = variance_table.columns.values
    df = variance_table.loc[:, cols[:-2]]
    hits = df.sum(axis = 1).sort_values(ascending = False) 
    cold_variance = hits[hits < cold_thresh].index.values
    return cold_variance

In [ ]:
def find_cold_variance_by_mutant(variance_table, mutant_name, cold_thresh):
    colds = find_cold_variance(variance_table,cold_thresh)
    variance_full, variance_genes = find_variance_by_mutant(variance_table, mutant_name) 
    var_mutant = variance_full.index.values
    intersection = list(set(colds).intersection(var_mutant))
    colds_mut = variance_full.loc[intersection, ['locus_tag', 'note']]
    return colds_mut.dropna()[['locus_tag', 'note']].drop_duplicates().sort_values(by = 'locus_tag')

In [ ]:
find_cold_variance_by_mutant(variance_table, 'W2', 3)

In [ ]:
# Read pathway cluster file 
pathways = pd.read_excel("G:\\Ye\\Evolution mutants\\Gene clusters in coelicolor.xlsx")
pathways.head(5)


In [ ]:
pathways_coe = pathways.drop(2).reset_index()

In [ ]:
locations = pathways_coe['Location'].apply(str).str.split('—')
short_inds = locations[locations.apply(len) == 1].index.values
for pos in short_inds:
    val = int(locations.iloc[pos][0])
    locations.iloc[pos] = [val, val]
    


In [ ]:
pathways_coe['location'] = locations
pathways_coe = pathways_coe.drop(columns = ['index', 'Location'])

In [ ]:
pathways_coe

In [ ]:
pathways_coe.loc[0,'location']

In [ ]:
variance_R1 = find_hot_variance_by_mutant(variance_table, 'R1', 1)
locus_R1 = variance_R1['locus_tag'].str.lstrip('SCO')
locus_ranges = pathways_coe['location']
for pos in locus_R1:
    pos = int(pos)
    for locus in locus_ranges:
        if pos in range(int(locus[0]), int(locus[1])):
            print(locus, pos)
          

In [ ]:
def find_pathway_hits(variance_table, mutant_name, hot_thresh, pathways_coe):
    variance_mut = find_hot_variance_by_mutant(variance_table, mutant_name, hot_thresh)
    locus_mut = variance_mut['locus_tag'].str.lstrip('SCO')
    locus_ranges = pathways_coe['location']
    for pos in locus_mut:
        pos = int(pos)
        for locus in locus_ranges:
            if pos in range(int(locus[0]), int(locus[1])):
                print(locus, pos)

In [ ]:
find_pathway_hits(variance_table, 'WS2', 1, pathways_coe)